In [1]:
import sys
import operator
import os.path
from IPython.core.debugger import Tracer

In [ ]:
advertiser="2997"
folder='..'
trainLogFile=os.path.join(folder,advertiser,'train.log.txt')
testLogFile=os.path.join(folder,advertiser,'test.log.txt')
trainYzxFile=os.path.join(folder,advertiser,'train.yzx.txt')
testYzxFile=os.path.join(folder,advertiser,'test.yzx.txt')
featIndexFile=os.path.join(folder,advertiser,'featindex.txt')

In [ ]:
oses = ["windows", "ios", "mac", "android", "linux"]
browsers = ["chrome", "sogou", "maxthon", "safari", "firefox", "theworld", "opera", "ie"]

f1s = ["weekday", "hour", "IP", "region", "city", "adexchange", "domain", "slotid", "slotwidth", "slotheight", "slotvisibility", "slotformat", "creative", "advertiser"]

f1sp = ["useragent", "slotprice"]

f2s = ["weekday,region"]

def featTrans(name, content):
    content = content.lower()
    if name == "useragent":
        operation = "other"
        for o in oses:
            if o in content:
                operation = o
                break
        browser = "other"
        for b in browsers:
            if b in content:
                browser = b
                break
        return operation + "_" + browser
    if name == "slotprice":
        price = int(content)
        if price > 100:
            return "101+"
        elif price > 50:
            return "51-100"
        elif price > 10:
            return "11-50"
        elif price > 0:
            return "1-10"
        else:
            return "0"

def getTags(content):
    if content == '\n' or len(content) == 0:
        return ["null"]
    return content.strip().split(',')

# initialize
namecol = {}
featindex = {}
maxindex = 0
fi = open(trainLogFile, 'r')
first = True

featindex['truncate'] = maxindex
maxindex += 1

for line in fi:
    s = line.split('\t')
    if first:
        first = False
        for i in range(0, len(s)):
            namecol[s[i].strip()] = i
            if i > 0:
                featindex[str(i) + ':other'] = maxindex
                maxindex += 1
        continue
    for f in f1s:
        col = namecol[f]
        content = s[col]
        feat = str(col) + ':' + content
        if feat not in featindex:
            featindex[feat] = maxindex
            maxindex += 1
    for f in f1sp:
        col = namecol[f]
        content = featTrans(f, s[col])
        feat = str(col) + ':' + content
        if feat not in featindex:
            featindex[feat] = maxindex
            maxindex += 1
    col = namecol["usertag"]
    tags = getTags(s[col])
    for tag in tags:
        feat = str(col) + ':' + tag
        if feat not in featindex:
            featindex[feat] = maxindex
            maxindex += 1

print 'feature size: ' + str(maxindex)
featvalue = sorted(featindex.iteritems(), key=operator.itemgetter(1))
fo = open(featIndexFile, 'w')
for fv in featvalue:
    fo.write(fv[0] + '\t' + str(fv[1]) + '\n')
fo.close()

# indexing train
print 'indexing ' + trainLogFile
fi = open(trainLogFile, 'r')
fo = open(trainYzxFile, 'w')

first = True
for line in fi:
    if first:
        first = False
        continue
    s = line.split('\t')
    fo.write(s[0] + ' ' + s[23]) # click + winning price
    index = featindex['truncate']
    fo.write(' ' + str(index) + ":1")
    for f in f1s: # every direct first order feature
        col = namecol[f]
        content = s[col]
        feat = str(col) + ':' + content
        if feat not in featindex:
            feat = str(col) + ':other'
        index = featindex[feat]
        fo.write(' ' + str(index) + ":1")
    for f in f1sp:
        col = namecol[f]
        content = featTrans(f, s[col])
        feat = str(col) + ':' + content
        if feat not in featindex:
            feat = str(col) + ':other'
        index = featindex[feat]
        fo.write(' ' + str(index) + ":1")
    col = namecol["usertag"]
    tags = getTags(s[col])
    for tag in tags:
        feat = str(col) + ':' + tag
        if feat not in featindex:
            feat = str(col) + ':other'
        index = featindex[feat]
        fo.write(' ' + str(index) + ":1")
    fo.write('\n')
fo.close()

# indexing test
print 'indexing ' + testLogFile
fi = open(testLogFile, 'r')
fo = open(testYzxFile, 'w')

first = True
for line in fi:
    if first:
        first = False
        continue
    s = line.split('\t')
    fo.write(s[0] + ' ' + s[23]) # click + winning price
    index = featindex['truncate']
    fo.write(' ' + str(index) + ":1")
    for f in f1s: # every direct first order feature
        col = namecol[f]
        if col >= len(s):
            print 'col: ' + str(col)
            print line
        content = s[col]
        feat = str(col) + ':' + content
        if feat not in featindex:
            feat = str(col) + ':other'
        index = featindex[feat]
        fo.write(' ' + str(index) + ":1")
    for f in f1sp:
        col = namecol[f]
        content = featTrans(f, s[col])
        feat = str(col) + ':' + content
        if feat not in featindex:
            feat = str(col) + ':other'
        index = featindex[feat]
        fo.write(' ' + str(index) + ":1")
    col = namecol["usertag"]
    tags = getTags(s[col])
    for tag in tags:
        feat = str(col) + ':' + tag
        if feat not in featindex:
            feat = str(col) + ':other'
        index = featindex[feat]
        fo.write(' ' + str(index) + ":1")
    fo.write('\n')
fo.close()


> <ipython-input-3-763b3329ea40>(77)<module>()
     75             maxindex += 1
     76     Tracer()()
---> 77     col = namecol["usertag"]
     78     tags = getTags(s[col])
     79     for tag in tags:

ipdb> l
     72         feat = str(col) + ':' + content
     73         if feat not in featindex:
     74             featindex[feat] = maxindex
     75             maxindex += 1
     76     Tracer()()
---> 77     col = namecol["usertag"]
     78     tags = getTags(s[col])
     79     for tag in tags:
     80         feat = str(col) + ':' + tag
     81         if feat not in featindex:
     82             featindex[feat] = maxindex

ipdb> p featindex
{'9:other': 9, '4:other': 4, '12:6e9c0e0bc0e584838ad187cad9852248': 33, '11:other': 11, '25:other': 25, '17:50': 36, '22:other': 22, '10:53': 31, '15:318719490': 34, '7:other': 7, '13:other': 13, '23:other': 23, '19:Na': 38, '25:2997': 40, '20:other': 20, '21:other': 21, '11:null': 32, '2:other': 2, '16:320': 35, '8:other': 8, '1:2': 27,